The below code iterates over the list of regions and fetches top video data per country stores it in a
json file in the top_videos folder, the json file contains severral attributes per videos including
date, location, list of tags (we can base links on common tags), engagement stats such as number of likes 
views and comments, channel,licensing info, category ID, language, title.
Note that some characters are in the following format: \u201cJ these are non english characters. they can still be used to create links between videos in foreign languages such as between spanish speaking countries. For this cell specifically you will needan authorization code. I have added your emails as authorized testers, when you run the cell you will be redicrected to the authorization code page where you can copy paste it. I already ran this cell and json files are saves in top_videos folder so you can skip.

In [ ]:
# -*- coding: utf-8 -*-
import os
import csv
import json
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret_1045102563277-cn5ih7mjh662130qb514n994d8uv5l23.apps.googleusercontent.com.json"
   
    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    
    # Create output folder if it doesn't exist
    output_folder = "top_videos"
    os.makedirs(output_folder, exist_ok=True)
    
    # Load country codes and names from CSV
    country_codes = []
    country_names = []
    with open("RegionCode.csv", "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row:  # Ensure the row is not empty
                country_codes.append(row[0])
                country_names.append(row[1])

    # Iterate over each country code and fetch the most popular videos with pagination
    for code, name in zip(country_codes, country_names):
        all_results = []  # To store all results for this country
        next_page_token = None  # Initialize next page token
        
        try:
            while True:
                # Make the API request with the current page token
                request = youtube.videos().list(
                    part="snippet,contentDetails,statistics",
                    chart="mostPopular",
                    regionCode=code,
                    maxResults=5,  # Adjust maxResults as needed
                    pageToken=next_page_token
                )
                response = request.execute()
                
                # Add the items from the current page to all_results
                all_results.extend(response['items'])
                
                # Check if there is a next page; if not, exit the loop
                next_page_token = response.get('nextPageToken')
                if not next_page_token:
                    break  # Exit loop if no more pages

            # Save all results to a JSON file
            filename = f"{name.replace(' ', '_')}_{code}.json"
            filepath = os.path.join(output_folder, filename)
            with open(filepath, "w") as json_file:
                json.dump(all_results, json_file, indent=4)
            print(f"Saved data for {name} to {filepath}")
        
        except HttpError as e:
            print(f"Failed to retrieve data for {name} ({code}). Skipping this region. Error: {e}")

if __name__ == "__main__":
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1045102563277-cn5ih7mjh662130qb514n994d8uv5l23.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=KJnqEvvdDyT5G9gucProtFZIgN6fCy&prompt=consent&access_type=offline


The below code fetches comments based on youtube URL, I haaven't implemented the loop yet to iteraate over top videos in each json file. here is the repo so you can install and reqs:

https://github.com/egbertbouman/youtube-comment-downloader

TO DO: fetch comments per video, clean output and store them so they can be analyze

In [1]:
from itertools import islice
from youtube_comment_downloader import *
downloader = YoutubeCommentDownloader()
comments = downloader.get_comments_from_url('https://www.youtube.com/watch?v=ALXzRaollc8', sort_by=SORT_BY_POPULAR)
for comment in islice(comments, 10):
    print(comment)

{'cid': 'UgwNxewgZ5PX72ihwNR4AaABAg', 'text': 'AFRICA ))))))))', 'time': 'for 3 uger siden', 'author': '@SAHNOUNSAADEDDINE', 'channel': 'UCZJnIcVeUd7LmqVc7h3h7Bg', 'votes': '26', 'replies': '', 'photo': 'https://yt3.ggpht.com/ytc/AIdro_m8XKHqSWkWTWqWfEqmk_VKzILNNjD_Lzb82OiQOJGQrEQgbhrNF_TgQFrd7LuMlqJAqQ=s88-c-k-c0x00ffffff-no-rj', 'heart': True, 'reply': False, 'time_parsed': 1729101085.747781}
{'cid': 'UgyoQWl9-7o75fCtL-F4AaABAg', 'text': 'Nice music❤❤❤', 'time': 'for 5 timer siden', 'author': '@rezasafari4676', 'channel': 'UC4krr4Xooga5GnA9700o1OQ', 'votes': '0', 'replies': '', 'photo': 'https://yt3.ggpht.com/l3AGuqSF4VKdHICKsUUMQsMAPW_5oAHhiFuV7Yb1fCjjo0R0I9kWqJu2rEc92jyD_wlrLpmc=s88-c-k-c0x00ffffff-no-rj', 'heart': False, 'reply': False, 'time_parsed': 1730901085.749621}
{'cid': 'Ugzle6m80q7NHE7ivWt4AaABAg', 'text': '16 min is TOP. Wow', 'time': 'for 2 dage siden', 'author': '@MV-sy4id', 'channel': 'UCJ8BxVBK0NEW7GO1nTNj6bw', 'votes': '5', 'replies': '', 'photo': 'https://yt3.ggpht

The below code is based on the following repo:

https://github.com/jdepoix/youtube-transcript-api/tree/master/youtube_transcript_api

It fetches a videos transcript in different available languages and also has the ability to translate them if needed. They are stored with timestamps which might not be relevant to us so we can extract just the text.

TO DO: iterate over videos with english captions and save them so they can be used for analysis

In [17]:
from youtube_transcript_api import YouTubeTranscriptApi

# retrieve the available transcripts
transcript_list = YouTubeTranscriptApi.list_transcripts('6dkDepLX0rk')

# iterate over all available transcripts
for transcript in transcript_list:

    # the Transcript object provides metadata properties
    # print(
    #     transcript.video_id,
    #     transcript.language,
    #     transcript.language_code,
    #     # whether it has been manually created or generated by YouTube
    #     transcript.is_generated,
    #     # whether this transcript can be translated or not
    #     transcript.is_translatable,
    #     # a list of languages the transcript can be translated to
    #     transcript.translation_languages,
    # )

    # fetch the actual transcript data
    print(transcript.fetch())

    # translating the transcript will return another transcript object
    print(transcript.translate('en').fetch())

# you can also directly filter for the language you are looking for, using the transcript list
transcript = transcript_list.find_transcript(['de', 'en'])  

# or just filter for manually created transcripts  
transcript = transcript_list.find_manually_created_transcript(['de', 'en'])  

# or automatically generated ones  
transcript = transcript_list.find_generated_transcript(['de', 'en'])

[{'text': '[choir singing]', 'start': 5.3, 'duration': 3.16}, {'text': '[piano plays]', 'start': 19.42, 'duration': 3.08}, {'text': "I'm not alone in feeling alone", 'start': 30.94, 'duration': 2.08}, {'text': 'One less thing to worry about', 'start': 33.7, 'duration': 2.04}, {'text': 'And if I counted all of us', 'start': 36.18, 'duration': 2.28}, {'text': "There'd be a lot", 'start': 40.46, 'duration': 1.36}, {'text': "Everything that's already\ncrossed my mind", 'start': 41.9, 'duration': 1.96}, {'text': 'Has already crossed their\nminds too', 'start': 44.5, 'duration': 2.04}, {'text': 'But despite all that, I feel\nalone', 'start': 46.9, 'duration': 2.32}, {'text': 'And so', 'start': 51.3, 'duration': 1.32}, {'text': "Sometimes I have suicidal\nthoughts,\nI'm not proud of them", 'start': 53.9, 'duration': 4.76}, {'text': 'Sometimes it feels like the\nonly way\nTo make them go away', 'start': 59.38, 'duration': 4.64}, {'text': 'These thoughts that make my\nlife\nA living hell', 'sta

NoTranscriptFound: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=6dkDepLX0rk! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['de', 'en']

For this video (6dkDepLX0rk) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")[TRANSLATABLE]

(GENERATED)
 - fr ("French (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - ab ("Abkhazian")
 - aa ("Afar")
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - ba ("Bashkir")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - br ("Breton")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da

TO DO: extract all the data and build the network

Suggested network:
- Nodes: top YouTube videos
- Attributes: 
    - Title/ topic
    - Description
    - Transcript
    - Location
    - Date
    - Engagement
    - Comments
    - tags
    - comments
    - transcript
    - category id
      
- Links: shared tags or categories (tags would lead to more connections) still not sure what categories are based on so we need to look that up in the youtube api
- Analysis:
  Modularity and Community Detection: Identify communities by location of similar or related videos, clustering by topics or sentiment patterns. Track trends in topics and sentiment based on comments, and video transcript analyzing how certain topics gain or lose popularity.
